In [30]:
import numpy as np
import torch.nn.functional as F
from torch.optim import Adam
import torch
import time
import random
import copy
import os

import sys


In [31]:
cwd = os.getcwd()
parent_dir = os.path.dirname(cwd)
sys.path.append(parent_dir)
from DataPipeline.preprocessing import node_encoder
from models import Model_GNNs
from torch_geometric.data import Data, Batch
import torch_geometric

In [32]:
def create_torch_graph_from_one_atom(atom, edge_size, encoding_option='charged'):
    num_atom = int(atom)

    atom_attribute = node_encoder(num_atom, encoding_option=encoding_option)
    # Create graph
    graph = torch_geometric.data.Data(x=atom_attribute.view(1, -1), edge_index=torch.empty((2, 0), dtype=torch.long), edge_attr=torch.empty((0, edge_size)))

    return graph

def sample_first_atom(encoding = 'charged'):
    if encoding == 'reduced' or encoding == 'charged':
        prob_dict = {'60': 0.7385023585929047, 
                    '80': 0.1000143018658728, 
                    '70': 0.12239949901813525, 
                    '90': 0.013786373862576426, 
                    '160': 0.017856330814654413,
                    '170': 0.007441135845856433}
    if encoding == 'polymer':
        prob_dict = {'60': 0.7489344573582472,
                    '70': 0.0561389266682314,
                    '80': 0.0678638375933265,
                    '160': 0.08724385192820308,
                    '90': 0.032130486119902095,
                    '140': 0.007666591133009364,
                    '150': 2.184919908044154e-05}

    
    return random.choices(list(prob_dict.keys()), weights=list(prob_dict.values()))[0]

In [33]:
batch_size = 5
edge_size = 3
true_graphs = [create_torch_graph_from_one_atom(sample_first_atom(), edge_size=edge_size, encoding_option='charged')
                            for i in range(batch_size)]

In [34]:
print(true_graphs[0].x.shape)
print(true_graphs[0].x)
print(true_graphs[0].edge_index)
print(true_graphs[0].edge_attr)

torch.Size([1, 13])
tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([], size=(2, 0), dtype=torch.int64)
tensor([], size=(0, 3))


In [35]:
a_value = torch.zeros(batch_size)
q_value = torch.zeros(batch_size)
pi_value = torch.zeros(batch_size)
queues = [[0] for i in range(batch_size)]

#not_graphs_finished is a mask of size batch size, initialized at true
not_mols_finished = [True for i in range(batch_size)]

#while not all atoms are finished
current_atoms = torch.zeros(batch_size)
for i in range(batch_size):
    if len(queues[i]) == 0:
        not_mols_finished[i] = False
        current_atoms[i]= -1 # precise no current_atoms
    else :
        current_atoms[i] = queues[i][0]

graph_for_gnn1 =[graph.clone() for graph in true_graphs]
#add feature position



In [ ]:
graph1.x = torch.cat([graph1.x, torch.zeros(graph1.x.size(0), 1)], dim=1)
for graph1 in graph_for_gnn1 :
        graph1.x[0:current_atoms[i], -1] = 1

In [40]:
liste = ['a', 'b', 'c', 'd', 'e', 'f']
indices_a_supprimer = [0, 0, 0]

# Tri de la liste d'indices dans l'ordre inverse
indices_a_supprimer.sort(reverse=True)

# Suppression des éléments
for indice in indices_a_supprimer:
    liste.pop(indice)

print(liste) 

['d', 'e', 'f']


In [41]:
liste = ['a', 'b', 'c', 'd', 'e', 'f']
indices_a_supprimer = [0, 1, 2]

# Tri de la liste d'indices dans l'ordre inverse
indices_a_supprimer.sort(reverse=True)

# Suppression des éléments
for indice in indices_a_supprimer:
    liste.pop(indice)

print(liste) 

['d', 'e', 'f']


In [39]:
liste = [0,1,2,3,4,5]
liste.pop(0)
print(liste)
print(liste[0])

[1, 2, 3, 4, 5]
1


In [36]:

#compute the gnn1 and apply softmax
batch_for_gnn1 = Batch.from_data_list(graph_for_gnn1)
batch_for_gnn1.to(device)
output_gnn1_q = torch.softmax(model_q.GNN1(batch_for_gnn1), dim = 1)
output_gnn1_a = torch.softmax(model_a.GNN1(batch_for_gnn1),  dim = 1)
output_gnn1_pi = torch.softmax(model_pi.GNN1(batch_for_gnn1), dim = 1)

#for each graph in the ouptut_gnn1_q we sample with a multinomial
next_step_gnn1 = torch.multinomial(output_gnn1_q, 1).item() #shape (batch_size, 1)

a_value = a_value * torch([output_gnn1_a[i][next_step_gnn1[i]] for i in range(batch_size) if not_mols_finished[i]])
pi_value = pi_value * torch([output_gnn1_pi[i][next_step_gnn1[i]] for i in range(batch_size) if not_mols_finished[i]])
q_value = q_value * torch([output_gnn1_q[i][next_step_gnn1[i]] for i in range(batch_size) if not_mols_finished[i]])

NameError: name 'device' is not defined